In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load dataset and preprocess
#pivot_df = pd.read_csv("synthetic_scenario_30_nodes.csv")
#pivot_df = pd.read_csv("simulated_office_environment.csv")
pivot_df = pd.read_csv("synthetic_temp_polling_data.csv")

pivot_df = pivot_df.apply(lambda x: x.fillna(x.mean()), axis=0)  # Fill missing values
pivot_df = pivot_df.head(10000)  # Restrict to first 20000 time steps

# Parameters
reward = 0.5
M = 2 # Maximum number of nodes that can be polled
theta = 0.5  # Threshold for reward condition
penalty = -0.5  # Penalty for polling when difference is <= theta
initial_value = 20  # Initial estimate for last polled values

# Set parameters
beta_1 = 0.9  # dEWMA parameter for state value
beta_2 = 0.01 # dEWMA parameter for rate of change

# Extract column names, ensuring "SN" is excluded
columns = [col for col in pivot_df.columns if col != "SN"]
num_nodes = len(columns)  # Number of sensor nodes based on dataset columns
num_time_steps = len(pivot_df)  # Total time steps based on dataset length

# Track the number of times each category is pulled
category_counts = {'Category A': 0, 'Category B': 0}

# Function to calculate Age of Incorrect Information (AoII) at the sink
def calculate_aoii_sink(current_time, last_received_time, last_rate_of_change):
    # Handle potential NaN or Inf values
    time_diff = current_time - last_received_time
    if np.isnan(last_rate_of_change) or np.isinf(last_rate_of_change):
        return 0.0  # Default to zero if rate of change is invalid
    return abs(time_diff * last_rate_of_change)

# Helper function to update state using dEWMA
def update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, delta_t, beta_1, beta_2):
    # Handle the case where delta_t is 0 to avoid division by zero
    if delta_t == 0:
        return measured_value, last_rate_of_change  # Return measured value and keep the last rate of change
    
    x1 = beta_1 * measured_value + (1 - beta_1) * (last_state_value + last_rate_of_change * delta_t)
    x2 = beta_2 * (x1 - last_state_value) / delta_t + (1 - beta_2) * last_rate_of_change
    return x1, x2

# Helper function to calculate reward
def calculate_reward(measured_value, last_state_value, theta, penalty):
    if abs(measured_value - last_state_value) > theta:
        return reward  # Reward
    return penalty  # Penalty

# Function to extract numeric node ID from column names dynamically, ensuring valid extraction
def extract_node_id(col_name):
    """Extract numeric node ID from column name, handling cases where no digits are found."""
    digits = ''.join(filter(str.isdigit, col_name))
    return int(digits) if digits else None  # Return None if no digits are found

# Dynamic Penalty Update algorithm based on Whittle indices
def dynamic_penalty_update(whittle_indices, M, current_lambda):
    # Convert dictionary values to a list of whittle indices, handling any NaN values
    c_values = []
    for v in whittle_indices.values():
        if np.isnan(v):
            c_values.append(-float('inf'))  # Treat NaN as negative infinity (won't be polled)
        else:
            c_values.append(v)
    
    # Identify the set ℰ of nodes where c_i > λ(t)
    eligible_nodes = [i for i, c_i in enumerate(c_values) if c_i > current_lambda]
    
    # If |ℰ| ≤ M, no penalty update needed
    if len(eligible_nodes) <= M:
        return current_lambda
    
    # Sort the c_i values in descending order
    sorted_c_values = sorted(c_values, reverse=True)
    
    # Identify the M-th value
    M_th_value = sorted_c_values[M-1]
    
    # Update penalty to the M-th value
    new_lambda = M_th_value
    
    return new_lambda

# Main function to simulate Whittle AoII with rewards and track transmission counts
def run_simulation_whittle_aoii_dynamic_penalty(pivot_df, columns, M, theta, penalty):
    cumulative_reward = 0  # Track total cumulative reward
    cumulative_rewards = []  # Store cumulative average reward over time
    last_update_times = {col: 0 for col in columns}  # Last update time for each node
    state_node = {col: np.array([20.0, 0.1]) for col in columns}  # Node states
    
    # Initialize dynamic penalty (λ) to 0
    aoii_penalty = 0.0
    
    # Track penalty evolution
    penalty_values = [aoii_penalty]
    
    # Track nodes polled at each time step
    nodes_polled_count = []
    
    # Set minimum timestamp difference to avoid division by zero
    min_delta_t = 1  # Minimum time difference of 1 to avoid division by zero

    for t in range(len(pivot_df)):
        # Step 1: Compute Whittle indices for each node based on AoII
        whittle_indices = {}
        for col in columns:
            last_state_value, last_rate_of_change = state_node[col]
            measured_value = pivot_df.loc[t, col]

            # Correct AoII calculation at the sink using rate of change
            current_aoii = calculate_aoii_sink(t, last_update_times[col], last_rate_of_change)
            future_aoii_passive = calculate_aoii_sink(t+1, last_update_times[col], last_rate_of_change)
            future_aoii_active = 0  # AoII resets to 0 if polled

            # Whittle index calculations - with safety checks for NaN/Inf values
            q_passive = current_aoii + future_aoii_passive
            q_active = current_aoii + future_aoii_active + aoii_penalty
            
            # Calculate the Whittle index with safeguards against invalid values
            whittle_index = q_passive - q_active
            
            # Handle NaN or Inf values
            if np.isnan(whittle_index) or np.isinf(whittle_index):
                whittle_indices[col] = -float('inf')  # Set to negative infinity if invalid
            else:
                whittle_indices[col] = whittle_index

        # Step 2: Update the dynamic penalty (λ) using the algorithm
        aoii_penalty = dynamic_penalty_update(whittle_indices, M, aoii_penalty)
        penalty_values.append(aoii_penalty)
        
        # Step 3: Select nodes to poll based on the updated penalty
        nodes_to_poll = [col for col in whittle_indices if whittle_indices[col] >= aoii_penalty]
        nodes_polled_count.append(len(nodes_to_poll))

        # Step 4: Poll selected nodes and calculate rewards
        for col in nodes_to_poll:
            measured_value = pivot_df.loc[t, col]
            last_state_value, last_rate_of_change = state_node[col]

            # Calculate reward after polling
            reward = calculate_reward(measured_value, last_state_value, theta, penalty)
            cumulative_reward += reward  # Update cumulative reward

            delta_t_dynamic = max(min_delta_t, t - last_update_times[col])  # Time since last update (ensure at least 1)

            # Update node state and last update time
            state_node[col] = update_node_state_dewma(
                measured_value, last_state_value, last_rate_of_change, delta_t_dynamic, beta_1=beta_1, beta_2=beta_2
            )
            last_update_times[col] = t

            # Extract node ID dynamically and categorize
            node_id = extract_node_id(col)
            if node_id is not None:
                if 1 <= node_id <= 5:
                    category_counts['Category A'] += 1
                elif 6 <= node_id <= 10:
                    category_counts['Category B'] += 1
             

        # Step 5: Calculate cumulative average reward
        cumulative_rewards.append(cumulative_reward / (t + 1))

    return cumulative_rewards, category_counts, penalty_values, nodes_polled_count

# Run the simulation with dynamic penalty
cumulative_rewards_whittle, category_pulled_counts, penalty_values, nodes_polled_count = run_simulation_whittle_aoii_dynamic_penalty(
    pivot_df, columns, M, theta, penalty
)

# Print the number of times each category was pulled
print("Transmission Count by Category:")
for category, count in category_pulled_counts.items():
    print(f"{category}: {count} times")

# Save cumulative rewards to CSV
pd.DataFrame(cumulative_rewards_whittle, columns=["cumulative_reward"]).to_csv(
    "cumulative_rewards_whittle_dynamic_penalty.csv", index=False
)



# Calculate statistics about the penalty values and nodes polled
avg_penalty = np.mean(penalty_values)
max_penalty = np.max(penalty_values)
min_penalty = np.min(penalty_values)
avg_nodes_polled = np.mean(nodes_polled_count)
max_nodes_polled = np.max(nodes_polled_count)

print(f"\nDynamic Penalty Statistics:")
print(f"Average Penalty: {avg_penalty:.4f}")
print(f"Maximum Penalty: {max_penalty:.4f}")
print(f"Minimum Penalty: {min_penalty:.4f}")
print(f"\nNodes Polled Statistics:")
print(f"Average Nodes Polled per Time Step: {avg_nodes_polled:.2f}")
print(f"Maximum Nodes Polled at any Time Step: {max_nodes_polled}")
print(f"Target Maximum (M): {M}")

Transmission Count by Category:
Category A: 124 times
Category B: 645 times

Dynamic Penalty Statistics:
Average Penalty: 19.4107
Maximum Penalty: 26.0190
Minimum Penalty: 0.0000

Nodes Polled Statistics:
Average Nodes Polled per Time Step: 0.08
Maximum Nodes Polled at any Time Step: 10
Target Maximum (M): 2


In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#pivot_df = pd.read_csv("synthetic_scenario_30_nodes.csv")
pivot_df = pd.read_csv("synthetic_temp_polling_data.csv")
pivot_df = pivot_df.head(20000)  # Restrict to first 20000 time steps


M = 2  # Maximum number of nodes that can be polled
aoii_penalty = 0.5
initial_value = 20  # Initial estimate for last polled values

# Set parameters
beta_1 = 0.95  # dEWMA parameter for state value
beta_2 = 0.1 # dEWMA parameter for rate of change

# Extract column names, ensuring "SN" is excluded
columns = [col for col in pivot_df.columns if col != "SN"]
num_nodes = len(columns)  # Number of sensor nodes based on dataset columns
num_time_steps = len(pivot_df)  # Total time steps based on dataset length

# Track the number of times each category is pulled
category_counts = {'Category A': 0, 'Category B': 0}

# Function to calculate Age of Incorrect Information (AoII) at the sink
def calculate_aoii_sink(current_time, last_received_time, last_rate_of_change):
    return abs((current_time - last_received_time) * last_rate_of_change)

# Helper function to update state using dEWMA
def update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, delta_t, beta_1, beta_2):
    x1 = beta_1 * measured_value + (1 - beta_1) * (last_state_value + last_rate_of_change * delta_t)
    x2 = beta_2 * (x1 - last_state_value) / delta_t + (1 - beta_2) * last_rate_of_change
    return x1, x2


# Function to extract numeric node ID from column names dynamically, ensuring valid extraction
def extract_node_id(col_name):
    """Extract numeric node ID from column name, handling cases where no digits are found."""
    digits = ''.join(filter(str.isdigit, col_name))
    return int(digits) if digits else None  # Return None if no digits are found

# Main function to simulate Whittle AoII with rewards and track transmission counts
def run_simulation_whittle_aoii(pivot_df, columns, M, aoii_penalty):
    
    last_update_times = {col: 0 for col in columns}  # Last update time for each node
    state_node = {col: np.array([20.0, 0.1]) for col in columns}  # Node states

    for t in range(len(pivot_df)):
        # Step 1: Compute Whittle indices for each node based on AoII
        whittle_indices = {}
        for col in columns:
            last_state_value, last_rate_of_change = state_node[col]
            measured_value = pivot_df.loc[t, col]

            # Correct AoII calculation at the sink using rate of change
            current_aoii = calculate_aoii_sink(t, last_update_times[col], last_rate_of_change)
            future_aoii_passive = calculate_aoii_sink(t+1, last_update_times[col], last_rate_of_change)
            future_aoii_active = 0  # AoII resets to 0 if polled

            # Whittle index calculations
            q_passive = current_aoii + future_aoii_passive
            q_active = current_aoii + future_aoii_active + aoii_penalty
            whittle_indices[col] = q_passive - q_active

        # Step 2: Select top M nodes to poll based on Whittle indices
        nodes_to_poll = [col for col in whittle_indices if whittle_indices[col] >= 0]
        if len(nodes_to_poll) > M:
            nodes_to_poll = sorted(nodes_to_poll, key=whittle_indices.get, reverse=True)[:M]

        # Step 3: Poll selected nodes and calculate rewards
        for col in nodes_to_poll:
            measured_value = pivot_df.loc[t, col]
            last_state_value, last_rate_of_change = state_node[col]

            delta_t_dynamic = t - last_update_times[col]  # Time since last update

            # Update node state and last update time
            state_node[col] = update_node_state_dewma(
                measured_value, last_state_value, last_rate_of_change, delta_t_dynamic, beta_1=beta_1, beta_2=beta_2
            )
            last_update_times[col] = t

            # Extract node ID dynamically and categorize
            node_id = extract_node_id(col)
            if node_id is not None:
                if 1 <= node_id <= 5:
                    category_counts['Category A'] += 1
                elif 6 <= node_id <= 10:
                    category_counts['Category B'] += 1


    return  category_counts

# Run the simulation
category_pulled_counts = run_simulation_whittle_aoii(
    pivot_df, columns, M, aoii_penalty
)

# Print the number of times each category was pulled
print("Transmission Count by Category:")
for category, count in category_pulled_counts.items():
    print(f"{category}: {count} times")




Transmission Count by Category:
Category A: 81 times
Category B: 403 times


In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load and restrict data
pivot_df = pd.read_csv("synthetic_temp_polling_data.csv").head(20000)
#pivot_df = pd.read_csv("synthetic_scenario_30_nodes_2.csv").head(20000)

# Parameters
M = 2  # Max nodes to poll simultaneously
aoii_penalty = 0.5
beta_1 = 0.1
beta_2 = 0.1

columns = [col for col in pivot_df.columns if col != "SN"]

# Initialize tracking variables
last_update_times = {col: 0 for col in columns}
state_node = {col: np.array([50.0, 0.1]) for col in columns}
temp_min_max_history = {col: {'min': float('inf'), 'max': float('-inf')} for col in columns}
category_counts = {'Category A': 0, 'Category B': 0, 'Category C': 0}

# Functions
def calculate_aoii_sink(current_time, last_received_time, normalised_rate_of_change):
    return abs((current_time - last_received_time) * normalised_rate_of_change)

def update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, delta_t, beta_1, beta_2):
    x1 = beta_1 * measured_value + (1 - beta_1) * (last_state_value + last_rate_of_change * delta_t)
    x2 = beta_2 * (x1 - last_state_value) / delta_t + (1 - beta_2) * last_rate_of_change
    return x1, x2

def extract_node_id(col_name):
    digits = ''.join(filter(str.isdigit, col_name))
    return int(digits) if digits else None

# Main simulation loop
for t in range(len(pivot_df)):
    whittle_indices = {}

    for col in columns:
        measured_value = pivot_df.loc[t, col]

        # Update min/max dynamically for normalisation
        temp_min_max_history[col]['min'] = min(temp_min_max_history[col]['min'], measured_value)
        temp_min_max_history[col]['max'] = max(temp_min_max_history[col]['max'], measured_value)
        min_temp = temp_min_max_history[col]['min']
        max_temp = temp_min_max_history[col]['max']

        epsilon = 1e-6  # avoid division by zero
        normalised_temp = (measured_value - min_temp) / (max_temp - min_temp + epsilon)

        # Calculate normalised rate of change
        if t > 0:
            prev_value = pivot_df.loc[t-1, col]
            prev_norm_temp = (prev_value - min_temp) / (max_temp - min_temp + epsilon)
            normalised_rate_of_change = abs(normalised_temp - prev_norm_temp)
        else:
            normalised_rate_of_change = 0

        current_aoii = calculate_aoii_sink(t, last_update_times[col], normalised_rate_of_change)
        future_aoii_passive = calculate_aoii_sink(t+1, last_update_times[col], normalised_rate_of_change)
        future_aoii_active = 0

        q_passive = current_aoii + future_aoii_passive
        q_active = current_aoii + future_aoii_active + aoii_penalty
        whittle_indices[col] = q_passive - q_active

    # Poll decision
    nodes_to_poll = [col for col in whittle_indices if whittle_indices[col] >= 0]
    if len(nodes_to_poll) > M:
        nodes_to_poll = sorted(nodes_to_poll, key=whittle_indices.get, reverse=True)[:M]

    for col in nodes_to_poll:
        measured_value = pivot_df.loc[t, col]
        last_state_value, last_rate_of_change = state_node[col]

        delta_t_dynamic = t - last_update_times[col]

        state_node[col] = update_node_state_dewma(
            measured_value, last_state_value, last_rate_of_change, delta_t_dynamic, beta_1, beta_2
        )
        last_update_times[col] = t

        # Track polling count by category
        node_id = extract_node_id(col)
        if node_id:
            if 1 <= node_id <= 5:
                category_counts['Category A'] += 1
            elif 6 <= node_id <= 10:
                category_counts['Category B'] += 1
            elif 21 <= node_id <= 30:
                category_counts['Category C'] += 1

# Display polling counts
print("Transmission Count by Category:")
for category, count in category_counts.items():
    print(f"{category}: {count} times")

Transmission Count by Category:
Category A: 14134 times
Category B: 11964 times
Category C: 0 times


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load and restrict data
pivot_df = pd.read_csv("synthetic_temp_polling_data.csv").head(20000)

# Parameters
M = 4  # Maximum nodes polled simultaneously
aoii_penalty = 0.3
beta_1 = 0.98
beta_2 = 0.98

columns = [col for col in pivot_df.columns if col != "SN"]

# Initialize tracking variables
last_update_times = {col: 0 for col in columns}
state_node = {col: np.array([20, 0.1]) for col in columns}  # Initialized as normalised
value_min_max_history = {col: {'min': float('inf'), 'max': float('-inf')} for col in columns}
category_counts = {'Category A': 0, 'Category B': 0, 'Category C': 0}

# Functions
def calculate_aoii_sink(current_time, last_received_time, normalised_rate_of_change):
    return abs((current_time - last_received_time) * normalised_rate_of_change)

def update_node_state_dewma(norm_measured_value, last_state_value, last_rate_of_change, delta_t, beta_1, beta_2):
    x1 = beta_1 * norm_measured_value + (1 - beta_1) * (last_state_value + last_rate_of_change * delta_t)
    x2 = beta_2 * (x1 - last_state_value) / delta_t + (1 - beta_2) * last_rate_of_change
    return x1, x2

def extract_node_id(col_name):
    digits = ''.join(filter(str.isdigit, col_name))
    return int(digits) if digits else None

# Main simulation loop
for t in range(len(pivot_df)):
    whittle_indices = {}

    for col in columns:
        last_state_value, last_rate_of_change = state_node[col]
        delta_t_dynamic = t - last_update_times[col]

        # Estimate the normalised state at sink (without new measurement)
        estimated_norm_value = last_state_value + last_rate_of_change * delta_t_dynamic
        normalised_rate_of_change = abs(last_rate_of_change)

        current_aoii = calculate_aoii_sink(t, last_update_times[col], normalised_rate_of_change)
        future_aoii_passive = calculate_aoii_sink(t + 1, last_update_times[col], normalised_rate_of_change)
        future_aoii_active = 0

        q_passive = current_aoii + future_aoii_passive
        q_active = current_aoii + future_aoii_active + aoii_penalty
        whittle_indices[col] = q_passive - q_active

    # Poll decision
    nodes_to_poll = [col for col in whittle_indices if whittle_indices[col] >= 0]
    if len(nodes_to_poll) > M:
        nodes_to_poll = sorted(nodes_to_poll, key=whittle_indices.get, reverse=True)[:M]

    for col in nodes_to_poll:
        measured_value = pivot_df.loc[t, col]

        # Dynamically update min/max and normalise measured value
        value_min_max_history[col]['min'] = min(value_min_max_history[col]['min'], measured_value)
        value_min_max_history[col]['max'] = max(value_min_max_history[col]['max'], measured_value)
        min_val = value_min_max_history[col]['min']
        max_val = value_min_max_history[col]['max']

        epsilon = 1e-6
        normalised_measurement = (measured_value - min_val) / (max_val - min_val + epsilon)

        last_state_value, last_rate_of_change = state_node[col]
        delta_t_dynamic = t - last_update_times[col]

        # Update state using normalised measurement
        state_node[col] = update_node_state_dewma(
            normalised_measurement, last_state_value, last_rate_of_change, delta_t_dynamic, beta_1, beta_2
        )
        last_update_times[col] = t

        # Track polling count by category
        node_id = extract_node_id(col)
        if node_id:
            if 1 <= node_id <= 5:
                category_counts['Category A'] += 1
            elif 6 <= node_id <= 10:
                category_counts['Category B'] += 1
            elif 21 <= node_id <= 30:
                category_counts['Category C'] += 1

# Display polling counts
print("Transmission Count by Category:")
for category, count in category_counts.items():
    print(f"{category}: {count} times")


Transmission Count by Category:
Category A: 64 times
Category B: 30 times
Category C: 0 times


In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load and restrict data
pivot_df = pd.read_csv("synthetic_temp_polling_data.csv").head(20000)

# Parameters
M = 2  # Maximum nodes polled simultaneously
aoii_penalty = 0.5
beta_1 = 0.9
beta_2 = 0.9

columns = [col for col in pivot_df.columns if col != "SN"]

# Initialize tracking variables
last_update_times = {col: 0 for col in columns}
state_node = {col: np.array([50, 0.1]) for col in columns}  # Initialised in normalised scale
value_min_max_history = {col: {'min': float('inf'), 'max': float('-inf')} for col in columns}
category_counts = {'Category A': 0, 'Category B': 0, 'Category C': 0}

# Functions
def calculate_aoii_sink(current_time, last_received_time, normalised_rate_of_change):
    return abs((current_time - last_received_time) * normalised_rate_of_change)

def update_node_state_dewma(normalised_measured_value, normalised_last_state_value, last_rate_of_change, delta_t, beta_1, beta_2):
    x1 = beta_1 * normalised_measurement + (1 - beta_1) * (last_state_value + last_rate_of_change * delta_t)
    x2 = beta_2 * (x1 - last_state_value) / delta_t + (1 - beta_2) * last_rate_of_change
    return x1, x2

def extract_node_id(col_name):
    digits = ''.join(filter(str.isdigit, col_name))
    return int(digits) if digits else None

# Main simulation loop
for t in range(len(pivot_df)):
    whittle_indices = {}

    for col in columns:
        last_state_value, last_rate_of_change = state_node[col]
        delta_t_dynamic = t - last_update_times[col]

        # Estimate normalised value at sink without new measurement
        estimated_norm_value = last_state_value + last_rate_of_change * delta_t_dynamic
        normalised_rate_of_change = abs(last_rate_of_change)

        current_aoii = calculate_aoii_sink(t, last_update_times[col], normalised_rate_of_change)
        future_aoii_passive = calculate_aoii_sink(t + 1, last_update_times[col], normalised_rate_of_change)
        future_aoii_active = 0

        q_passive = current_aoii + future_aoii_passive
        q_active = current_aoii + future_aoii_active + aoii_penalty
        whittle_indices[col] = q_passive - q_active

    # Poll decision
    nodes_to_poll = [col for col in whittle_indices if whittle_indices[col] >= 0]
    if len(nodes_to_poll) > M:
        nodes_to_poll = sorted(nodes_to_poll, key=whittle_indices.get, reverse=True)[:M]

    for col in nodes_to_poll:
        measured_value = pivot_df.loc[t, col]

        # Update min/max and normalise measured value upon receiving
        value_min_max_history[col]['min'] = min(value_min_max_history[col]['min'], measured_value)
        value_min_max_history[col]['max'] = max(value_min_max_history[col]['max'], measured_value)
        min_val = value_min_max_history[col]['min']
        max_val = value_min_max_history[col]['max']

        epsilon = 1e-6
        normalised_measurement = (measured_value - min_val) / (max_val - min_val + epsilon)

        last_state_value, last_rate_of_change = state_node[col]
        delta_t_dynamic = t - last_update_times[col]

        # Update state using normalised values
        state_node[col] = update_node_state_dewma(
            normalised_measurement, last_state_value, last_rate_of_change, delta_t_dynamic, beta_1, beta_2
        )
        last_update_times[col] = t

        # Track polling count by category
        node_id = extract_node_id(col)
        if node_id:
            if 1 <= node_id <= 5:
                category_counts['Category A'] += 1
            elif 6 <= node_id <= 10:
                category_counts['Category B'] += 1
            elif 21 <= node_id <= 30:
                category_counts['Category C'] += 1

# Display polling counts
print("Transmission Count by Category:")
for category, count in category_counts.items():
    print(f"{category}: {count} times")


Transmission Count by Category:
Category A: 67 times
Category B: 43 times
Category C: 0 times


In [26]:
# Add this adaptive threshold approach to your existing implementation

# Function to calculate adaptive thresholds based on node variance
def calculate_adaptive_threshold(values, base_theta, min_factor=0.5, max_factor=2.0):
    """
    Calculate an adaptive threshold based on the variability of the sensor.
    
    Parameters:
    - values: List of historical values for the sensor
    - base_theta: Base threshold value (θ)
    - min_factor: Minimum scaling factor
    - max_factor: Maximum scaling factor
    
    Returns:
    - Adaptive threshold value
    """
    if len(values) < 2:
        return base_theta
    
    # Calculate coefficient of variation (CV)
    # CV = std / mean, which gives a normalized measure of dispersion
    mean = np.mean(values)
    std = np.std(values)
    
    # Avoid division by zero
    if abs(mean) < 1e-10:
        cv = 1.0  # Default to 1.0 if mean is near zero
    else:
        cv = std / abs(mean)
    
    # Scale threshold inversely with CV
    # Higher variability (higher CV) -> lower threshold (more sensitive)
    # Lower variability (lower CV) -> higher threshold (less sensitive)
    scale_factor = 1.0 / max(cv, 0.1)  # Limit the maximum scale factor
    
    # Clamp the scale factor within reasonable bounds
    scale_factor = max(min_factor, min(max_factor, scale_factor))
    
    return base_theta * scale_factor

# Modified reward calculation function with adaptive thresholds
def calculate_reward_adaptive(measured_value, last_state_value, node_adaptive_thresholds, col, base_penalty):
    """
    Calculate reward using adaptive thresholds.
    
    Parameters:
    - measured_value: Current measured value
    - last_state_value: Previous state value
    - node_adaptive_thresholds: Dictionary of adaptive thresholds for each node
    - col: Column name (node identifier)
    - base_penalty: Base penalty value
    
    Returns:
    - Reward or penalty value
    """
    # Get the adaptive threshold for this node
    adaptive_theta = node_adaptive_thresholds[col]
    
    # Calculate reward based on adaptive threshold
    if abs(measured_value - last_state_value) > adaptive_theta:
        return reward  # Reward is kept constant
    return base_penalty  # Use the base penalty

# Function to initialize and manage adaptive thresholds
def initialize_adaptive_thresholds(pivot_df, columns, base_theta, window_size=100):
    """
    Initialize adaptive thresholds for each node.
    
    Parameters:
    - pivot_df: DataFrame containing sensor data
    - columns: List of column names representing sensor nodes
    - base_theta: Base threshold value (θ)
    - window_size: Size of the window for calculating statistics
    
    Returns:
    - Dictionary of adaptive thresholds and window values for each node
    """
    adaptive_info = {}
    for col in columns:
        # Use first window_size readings (or all if less) to initialize
        initial_window_size = min(window_size, len(pivot_df))
        values = pivot_df.loc[0:initial_window_size-1, col].values.tolist()
        
        # Calculate initial adaptive threshold
        adaptive_theta = calculate_adaptive_threshold(values, base_theta)
        
        adaptive_info[col] = {
            'threshold': adaptive_theta,
            'window': values,
            'window_size': window_size
        }
    
    return adaptive_info

# Function to update adaptive thresholds with new measurements
def update_adaptive_thresholds(adaptive_info, col, new_value, base_theta):
    """
    Update adaptive thresholds with new measurements.
    
    Parameters:
    - adaptive_info: Dictionary containing threshold information
    - col: Column name (node identifier)
    - new_value: New measured value
    - base_theta: Base threshold value (θ)
    
    Returns:
    - Updated adaptive_info dictionary
    """
    window = adaptive_info[col]['window']
    window_size = adaptive_info[col]['window_size']
    
    # Add new value and remove oldest if window is full
    window.append(new_value)
    if len(window) > window_size:
        window.pop(0)
    
    # Recalculate adaptive threshold
    adaptive_info[col]['threshold'] = calculate_adaptive_threshold(window, base_theta)
    
    return adaptive_info

# Modified main simulation function with adaptive thresholds
def run_simulation_whittle_aoii_adaptive(pivot_df, columns, M, base_theta, base_penalty, adaptive_window_size=100):
    cumulative_reward = 0
    cumulative_rewards = []
    last_update_times = {col: 0 for col in columns}
    state_node = {col: np.array([pivot_df.loc[0, col], 0.1]) for col in columns}
    
    # Initialize adaptive thresholds
    adaptive_info = initialize_adaptive_thresholds(pivot_df, columns, base_theta, adaptive_window_size)
    
    # Extract just the thresholds for easier access
    node_adaptive_thresholds = {col: adaptive_info[col]['threshold'] for col in columns}
    
    # Initialize category counts
    category_counts = {'Category A': 0, 'Category B': 0}
    
    # Initialize dynamic penalty
    aoii_penalty = 0.0
    penalty_values = [aoii_penalty]
    nodes_polled_count = []
    min_delta_t = 1

    for t in range(len(pivot_df)):
        # Calculate Whittle indices (standard calculation)
        whittle_indices = {}
        for col in columns:
            last_state_value, last_rate_of_change = state_node[col]
            measured_value = pivot_df.loc[t, col]

            current_aoii = calculate_aoii_sink(t, last_update_times[col], last_rate_of_change)
            future_aoii_passive = calculate_aoii_sink(t+1, last_update_times[col], last_rate_of_change)
            future_aoii_active = 0

            q_passive = current_aoii + future_aoii_passive
            q_active = current_aoii + future_aoii_active + aoii_penalty
            
            whittle_index = q_passive - q_active
            
            if np.isnan(whittle_index) or np.isinf(whittle_index):
                whittle_indices[col] = -float('inf')
            else:
                whittle_indices[col] = whittle_index

        # Update dynamic penalty
        aoii_penalty = dynamic_penalty_update(whittle_indices, M, aoii_penalty)
        penalty_values.append(aoii_penalty)
        
        # Select nodes to poll
        nodes_to_poll = [col for col in whittle_indices if whittle_indices[col] >= aoii_penalty]
        nodes_polled_count.append(len(nodes_to_poll))

        # Poll selected nodes with adaptive thresholds for rewards
        for col in nodes_to_poll:
            measured_value = pivot_df.loc[t, col]
            last_state_value, last_rate_of_change = state_node[col]

            # Calculate reward using adaptive threshold
            reward_value = calculate_reward_adaptive(
                measured_value, last_state_value, node_adaptive_thresholds, col, base_penalty
            )
            cumulative_reward += reward_value

            delta_t_dynamic = max(min_delta_t, t - last_update_times[col])

            # Update node state (standard update)
            state_node[col] = update_node_state_dewma(
                measured_value, last_state_value, last_rate_of_change, delta_t_dynamic, beta_1, beta_2
            )
            last_update_times[col] = t
            
            # Update adaptive thresholds with new measurement
            adaptive_info = update_adaptive_thresholds(adaptive_info, col, measured_value, base_theta)
            node_adaptive_thresholds[col] = adaptive_info[col]['threshold']

            # Categorize based on node ID
            node_id = extract_node_id(col)
            if node_id is not None:
                if 1 <= node_id <= 5:
                    category_counts['Category A'] += 1
                elif 6 <= node_id <= 10:
                    category_counts['Category B'] += 1

        # Calculate cumulative reward
        cumulative_rewards.append(cumulative_reward / (t + 1))

    return cumulative_rewards, category_counts, penalty_values, nodes_polled_count, node_adaptive_thresholds

# Run the simulation with adaptive thresholds
cumulative_rewards_adaptive, category_pulled_counts, penalty_values, nodes_polled_count, final_thresholds = run_simulation_whittle_aoii_adaptive(
    pivot_df, columns, M, theta, penalty
)

# Print the adaptive thresholds at the end of simulation
print("Final Adaptive Thresholds:")
for col, threshold in final_thresholds.items():
    print(f"{col}: {threshold:.4f}")

# Compare category counts between different approaches
print("\nCategory Counts Comparison:")
print(f"Adaptive Thresholds: {category_pulled_counts}")

Final Adaptive Thresholds:
node1: 1.0000
node2: 1.0000
node3: 1.0000
node4: 1.0000
node5: 1.0000
node6: 1.0000
node7: 1.0000
node8: 1.0000
node9: 1.0000
node10: 1.0000

Category Counts Comparison:
Adaptive Thresholds: {'Category A': 521, 'Category B': 515}


In [49]:
import pandas as pd
import numpy as np

# Load the dataset
pivot_df = pd.read_csv("synthetic_temp_polling_data.csv")
pivot_df = pivot_df.head(20000)  # Restrict to first 20000 time steps

M = 2  # Maximum number of nodes that can be polled
aoii_penalty = 0.5
initial_value = 20  # Initial estimate for last polled values

# Set parameters
beta_1 = 0.95  # dEWMA parameter for state value
beta_2 = 0.99   # dEWMA parameter for rate of change

# Extract column names, ensuring "SN" is excluded
columns = [col for col in pivot_df.columns if col != "SN"]

# Function to calculate Age of Incorrect Information (AoII) at the sink
def calculate_aoii_sink(current_time, last_received_time, last_rate_of_change):
    return abs((current_time - last_received_time) * last_rate_of_change)

# Helper function to update state using dEWMA
def update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, delta_t, beta_1, beta_2):
    x1 = beta_1 * measured_value + (1 - beta_1) * (last_state_value + last_rate_of_change * delta_t)
    x2 = beta_2 * (x1 - last_state_value) / delta_t + (1 - beta_2) * last_rate_of_change
    return x1, x2

# Function to extract numeric node ID from column names
def extract_node_id(col_name):
    digits = ''.join(filter(str.isdigit, col_name))
    return int(digits) if digits else None

# Normalize a value using fixed min-max range for each category
def normalize_value(value, node_id):
    # Fixed ranges per category
    if 1 <= node_id <= 5:  # Category A
        min_val = 15
        max_val = 25
    elif 6 <= node_id <= 10:  # Category B
        min_val = 75
        max_val = 125
    else:
        # Default fallback
        return value / 100.0
    
    # Standard min-max normalization with clipping
    normalized = (value - min_val) / (max_val - min_val)
    normalized = max(0, min(1, normalized))
    
    return normalized

# Run normalized simulation
def run_normalized_simulation():
    # Initialize tracking variables
    last_update_times = {col: 0 for col in columns}
    state_node = {}
    for col in columns:
        node_id = extract_node_id(col)
        if node_id is not None:
            # Get initial value from dataset or use default
            initial_val = pivot_df.loc[0, col] if not pd.isna(pivot_df.loc[0, col]) else initial_value
            normalized_val = normalize_value(initial_val, node_id)
            state_node[col] = np.array([normalized_val, 0.1])
        else:
            state_node[col] = np.array([0.5, 0.1])
    
    # Track category counts
    category_counts = {'Category A': 0, 'Category B': 0}

    for t in range(len(pivot_df)):
        # Progress indicator (every 5000 steps)
        if t % 5000 == 0:
            print(f"Step {t}/{len(pivot_df)}...")
            
        # Compute Whittle indices for each node
        whittle_indices = {}
        for col in columns:
            node_id = extract_node_id(col)
            if node_id is None:
                continue
                
            last_state_value, last_rate_of_change = state_node[col]
            measured_value = pivot_df.loc[t, col]
            
            # Skip if value is NaN
            if pd.isna(measured_value):
                continue

            # Calculate AoII in normalized space
            current_aoii = calculate_aoii_sink(t, last_update_times[col], last_rate_of_change)
            future_aoii_passive = calculate_aoii_sink(t+1, last_update_times[col], last_rate_of_change)
            future_aoii_active = 0  # AoII resets to 0 if polled

            # Whittle index calculations
            q_passive = current_aoii + future_aoii_passive
            q_active = current_aoii + future_aoii_active + aoii_penalty
            whittle_indices[col] = q_passive - q_active

        # Select nodes to poll based on Whittle indices
        nodes_to_poll = [col for col in whittle_indices if whittle_indices[col] >= 0]
        if len(nodes_to_poll) > M:
            nodes_to_poll = sorted(nodes_to_poll, key=whittle_indices.get, reverse=True)[:M]
            
        # Poll selected nodes and update states
        for col in nodes_to_poll:
            measured_value = pivot_df.loc[t, col]
            
            # Skip if value is NaN
            if pd.isna(measured_value):
                continue
            
            node_id = extract_node_id(col)
            if node_id is None:
                continue
                
            # Normalize the measured value
            normalized_value = normalize_value(measured_value, node_id)
            
            # Get last state in normalized space
            last_state_value, last_rate_of_change = state_node[col]
            
            # Calculate time since last update (min 1 to avoid division by zero)
            delta_t_dynamic = max(1, t - last_update_times[col])
            
            # Update state using dEWMA with normalized measured value
            state_node[col] = update_node_state_dewma(
                normalized_value, last_state_value, last_rate_of_change, 
                delta_t_dynamic, beta_1, beta_2
            )
            
            last_update_times[col] = t

            # Categorize based on node ID
            if 1 <= node_id <= 5:
                category_counts['Category A'] += 1
            elif 6 <= node_id <= 10:
                category_counts['Category B'] += 1

    return category_counts

# Run the simulation and print results
print("Running normalized simulation...")
category_counts = run_normalized_simulation()

print("\nRESULTS:")
print(f"Category A: {category_counts['Category A']} polls")
print(f"Category B: {category_counts['Category B']} polls")

Running normalized simulation...
Step 0/20000...
Step 5000/20000...
Step 10000/20000...
Step 15000/20000...

RESULTS:
Category A: 72 polls
Category B: 14 polls
